#### 1.3.1
Please make use of the data you have processed in the `1.2` to build a sample ranking model.
> We understand that the best model is not built in a day with limited features. Thus this task is only evaluating on your coding abilities and will not evaluate much on modelling abilities thus you do not need to spend too much time to optimize the model to get best performance